In [1]:
import numpy as np
from __future__ import print_function
import keras
from keras.layers import Dense,Input,Lambda
from keras.models import Model
from keras.optimizers import SGD
import tensorflow as tf
import keras.backend as K

Using TensorFlow backend.


In [2]:
import struct

def _read_int(buf):
    return struct.unpack('>i', buf.read(4))[0]

def load_idx(filepath):
    with open(filepath, 'rb') as f:
        magic = _read_int(f)
        if magic == 2051:
            shape = (_read_int(f), _read_int(f), _read_int(f))
        elif magic == 2049:
            shape = _read_int(f)
        else:
            raise RuntimeError('could not parse header correctly')
        array = np.fromfile(f, dtype='B', count=np.prod(shape))
        array = np.reshape(array, shape)
    return array

In [3]:
pat_file = './infimnist/data/t10k-images-idx3-ubyte'
lab_file = './infimnist/data/t10k-labels-idx1-ubyte'
test_x, test_y = map(load_idx, [pat_file, lab_file])
pat_file = './infimnist/data/train-images-idx3-ubyte'
lab_file = './infimnist/data/train-labels-idx1-ubyte'
train_x, train_y = map(load_idx, [pat_file, lab_file])

In [4]:
train_x.shape

(60000, 28, 28)

In [5]:
test_x.shape

(10000, 28, 28)

In [6]:
#train_x = np.reshape(train_x,(60000,784))

In [7]:
#test_x = np.reshape(test_x,(10000,784))

In [42]:
# Data Generator for generating numbers to be summed up. 'N_' is the number of digits to generate for a given batch.
class DataGenerator(object):
    
    def __init__(self, batch_size = 32, shuffle = True):
        self.batch_size = batch_size
        self.shuffle = shuffle

    def generate(self,N_=None):
        
         while 1:
            if N_ == None: #if N_ is not specified, randomly choose N per batch.
                N = (np.random.randint(10, size=1)+1)[0]
            else:
                N = N_
            rand_values = np.random.randint(60000,size=(N*self.batch_size))
            
            X = train_x[rand_values]
            X = np.reshape(X,(self.batch_size,N,28,28))
            
            y = train_y[rand_values]
            y = np.reshape(y,(self.batch_size,N,1))
            y = np.sum(y,axis=1)
            y = np.reshape(y,(self.batch_size,1))
            
            yield X, y

In [43]:
train_x[[1,2]].shape

(2, 28, 28)

In [45]:
# Parameters
params = {'batch_size': 32,
          'shuffle': True}

In [46]:
train_generator = DataGenerator(**params).generate()
test_generator = DataGenerator(**params).generate(N_=100)

In [47]:
a = train_generator.next()

In [48]:
a[0].shape

(32, 4, 28, 28)

In [49]:
a[1][0]

array([17], dtype=uint64)

In [50]:
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv3D, MaxPooling3D, Reshape

x = Input(shape=(None,28,28,1))
y = Conv3D(32, (3,3,3), activation='relu',padding='same')(x)
y = Conv3D(64, (3,3,3), activation='relu',padding='same')(y)
y = MaxPooling3D(pool_size=(2,2,2))(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = MaxPooling3D(pool_size=(2,2,2))(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = MaxPooling3D(pool_size=(2,2,2))(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = MaxPooling3D(pool_size=(2,2,2))(y)

#y = Dropout(0.25)(y)
#y = Flatten()(y)

# Summing the representations outputted by \phi Network.
y = Lambda(lambda x_: tf.reduce_sum(x_,axis=[1,2,3,4],keep_dims=True))(y)
y = Reshape((1,))(y)

# \rho Network
y = Dense(64, activation='relu')(y)
y = Dense(32, activation='relu')(y)
y = Dense(16, activation='relu')(y)
#y = Dense(128, activation='relu')(y)
#y = Dense(128, activation='relu')(y)
y = Dense(1,activation='relu')(y)

model = Model(x,y)


In [53]:
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Reshape

x = Input(shape=(None,28,28))
y = Conv2D(32, (3,3), activation='relu',padding='same')(x)
y = Conv2D(64, (3,3), activation='relu',padding='same')(y)
y = MaxPooling2D(pool_size=(2,2))(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = MaxPooling3D(pool_size=(2,2,2))(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = MaxPooling3D(pool_size=(2,2,2))(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = Conv3D(32, (3,3,3), activation='relu',padding='same')(y)
# y = MaxPooling3D(pool_size=(2,2,2))(y)

#y = Dropout(0.25)(y)
#y = Flatten()(y)

# Summing the representations outputted by \phi Network.
y = Lambda(lambda x_: tf.reduce_sum(x_,axis=[1,2,3],keep_dims=True))(y)
y = Reshape((1,))(y)

# \rho Network
y = Dense(64, activation='relu')(y)
y = Dense(32, activation='relu')(y)
y = Dense(16, activation='relu')(y)
#y = Dense(128, activation='relu')(y)
#y = Dense(128, activation='relu')(y)
y = Dense(1,activation='relu')(y)

model = Model(x,y)


In [54]:
for layer in model.layers:
    print (layer.name,layer.output_shape,layer.output)

input_9 (None, None, 28, 28) Tensor("input_9:0", shape=(?, ?, 28, 28), dtype=float32)
conv2d_13 (None, None, 28, 32) Tensor("conv2d_13/Relu:0", shape=(?, ?, 28, 32), dtype=float32)
conv2d_14 (None, None, 28, 64) Tensor("conv2d_14/Relu:0", shape=(?, ?, 28, 64), dtype=float32)
max_pooling2d_6 (None, None, 14, 64) Tensor("max_pooling2d_6/MaxPool:0", shape=(?, ?, 14, 64), dtype=float32)
lambda_6 (None, 1, 1, 1) Tensor("lambda_6/Sum:0", shape=(?, 1, 1, 1), dtype=float32)
reshape_4 (None, 1) Tensor("reshape_4/Reshape:0", shape=(?, 1), dtype=float32)
dense_13 (None, 64) Tensor("dense_13/Relu:0", shape=(?, 64), dtype=float32)
dense_14 (None, 32) Tensor("dense_14/Relu:0", shape=(?, 32), dtype=float32)
dense_15 (None, 16) Tensor("dense_15/Relu:0", shape=(?, 16), dtype=float32)
dense_16 (None, 1) Tensor("dense_16/Relu:0", shape=(?, 1), dtype=float32)


In [55]:
train_steps_per_epoch=100000
test_steps_per_epoch=10000
train_epochs=10

In [56]:
def least_squared_error(y_true, y_pred):
    return K.sum(K.square(y_pred - y_true), axis=-1)

In [57]:
model.compile(optimizer='adam',
              loss=least_squared_error,
              metrics=["mse",least_squared_error,'accuracy'])

In [ ]:
model.fit_generator(train_generator,
                    steps_per_epoch=train_steps_per_epoch,
                    epochs=train_epochs,
                    validation_data=test_generator,
                    validation_steps=test_steps_per_epoch
                    )

Epoch 1/10
  6877/100000 [=>............................] - ETA: 2965s - loss: 826.7776 - mean_squared_error: 826.7776 - least_squared_error: 826.7776 - acc: 0.0103

In [ ]:
a = np.random.randint(10, size=(100))
model.predict(np.reshape(a,(1,100,1)))

In [ ]:
np.sum(a)

In [ ]:
X = np.random.randint(10, size=(10*32))
X = np.reshape(X,(32,10))

In [ ]:
model.evaluate_generator(DataGenerator(**params).generate(N=13),steps=1000)

In [ ]:
X.shape

In [ ]:
X[0]